In [3]:
import pandas as pd         
import numpy as np           
from sklearn.model_selection import train_test_split  
from sklearn.metrics import mean_squared_error  

In [1]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")  

In [4]:
# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 16 1330k   16  222k    0     0   555k      0  0:00:02 --:--:--  0:00:02  558k
100 1330k  100 1330k    0     0  1035k      0  0:00:01  0:00:01 --:--:-- 1038k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 87 1253k   87 1101k    0     0  1034k      0  0:00:01  0:00:01 --:--:-- 1037k
100 1253k  100 1253k    0     0  1053k      0  0:00:01  0:00:01 --:--:-- 1055k


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import pickle
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [6]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [7]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [44]:
df_val.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration,PU_DO
0,2,2024-02-01 00:15:53,2024-02-01 00:24:20,N,1.0,75,161,1.0,2.77,13.5,...,3.75,0.0,NaN,1.0,22.50,1.0,1.0,2.75,8.450000,75_161
1,2,2024-01-31 22:59:22,2024-01-31 23:27:14,N,1.0,43,242,1.0,7.76,36.6,...,0.00,0.0,NaN,1.0,39.10,2.0,1.0,0.00,27.866667,43_242
2,2,2024-02-01 00:30:29,2024-02-01 00:35:32,N,1.0,75,238,1.0,1.03,7.9,...,2.60,0.0,NaN,1.0,13.00,1.0,1.0,0.00,5.050000,75_238
3,2,2024-01-31 23:56:42,2024-02-01 00:06:53,N,1.0,80,17,2.0,1.36,11.4,...,4.17,0.0,NaN,1.0,18.07,1.0,1.0,0.00,10.183333,80_17
5,2,2024-02-01 00:06:23,2024-02-01 00:10:10,N,1.0,97,25,1.0,0.34,5.1,...,0.00,0.0,NaN,1.0,7.60,2.0,1.0,0.00,3.783333,97_25


In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## Para el modelo Gradient Boosting:

In [16]:
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingRegressor

# Iniciar el experimento
mlflow.set_experiment("Gradient Boosting Experiment")


2024/09/20 16:05:23 INFO mlflow.tracking.fluent: Experiment with name 'Gradient Boosting Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/92863f65ec68402babfe2d09ddebc317', creation_time=1726869923235, experiment_id='1', last_update_time=1726869923235, lifecycle_stage='active', name='Gradient Boosting Experiment', tags={}>

In [17]:
with mlflow.start_run():
    # Entrenar el modelo
    gb_model = GradientBoostingRegressor()
    gb_model.fit(X_train, y_train)

    # Hacer predicciones
    y_pred = gb_model.predict(X_val)

    # Calcular el RMSE
    rmse_value = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"RMSE: {rmse_value}")

    # Registrar el modelo y la métrica en MLflow
    mlflow.sklearn.log_model(gb_model, "gradient-boosting-model")
    mlflow.log_metric("rmse", rmse_value)

2024/09/20 16:05:36 WARNING mlflow.data.pandas_dataset: Failed to infer schema for Pandas dataset. Exception: Failed to infer schema for pandas.Series 0        <Compressed Sparse Row sparse matrix of dtype ...
1        <Compressed Sparse Row sparse matrix of dtype ...
2        <Compressed Sparse Row sparse matrix of dtype ...
3        <Compressed Sparse Row sparse matrix of dtype ...
4        <Compressed Sparse Row sparse matrix of dtype ...
                               ...                        
54368    <Compressed Sparse Row sparse matrix of dtype ...
54369    <Compressed Sparse Row sparse matrix of dtype ...
54370    <Compressed Sparse Row sparse matrix of dtype ...
54371    <Compressed Sparse Row sparse matrix of dtype ...
54372    <Compressed Sparse Row sparse matrix of dtype ...
Name: 0, Length: 54373, dtype: object. Error: Data <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (1, 5093)>
  Coords	Values
  (0, 2065)	1.0
  (0, 5092)	1.98

ValueError: setting an array element with a sequence.

## Para el modelo Random Forest:

In [24]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor

# Iniciar el experimento
mlflow.set_experiment("Random Forest Experiment")

with mlflow.start_run():
    # Entrenar el modelo
    rf_model = RandomForestRegressor()
    rf_model.fit(X_train, y_train)

    # Registrar el modelo y su métrica
    mlflow.sklearn.log_model(rf_model, "random-forest-model")
    mlflow.log_metric("rmse", rmse_value)


2024/09/20 14:28:08 INFO mlflow.tracking.fluent: Experiment with name 'Random Forest Experiment' does not exist. Creating a new experiment.
2024/09/20 14:32:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Buscar hiperparámetros:

In [25]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 20]}
grid_search = GridSearchCV(GradientBoostingRegressor(), param_grid, scoring="neg_mean_squared_error")
grid_search.fit(X_train, y_train)

# Guardar el mejor modelo
best_model = grid_search.best_estimator_
mlflow.sklearn.log_model(best_model, "best-gb-model")


2024/09/20 14:58:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## OTRO INTENTO PARA LOS MODELOS

In [32]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
import numpy as np
import pathlib
import pickle

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def objective_rf(params):
    params['max_depth'] = int(params['max_depth'])  # Convertir a int aquí
    params['min_samples_split'] = int(params['min_samples_split'])  
    with mlflow.start_run(nested=True):
        mlflow.log_params(params)
        
        # Crea y entrena el modelo
        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)

        # Predicción en el conjunto de validación
        y_pred = rf.predict(X_val)

        # Calcular RMSE
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}



In [34]:
# Espacio de búsqueda de hiperparámetros para Random Forest
search_space_rf = {
    'n_estimators': hp.choice('n_estimators', [50, 100, 150]),
    'max_depth': hp.quniform('max_depth', 5, 30,1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
    'random_state': 42
}

# Optimización con Hyperopt
with mlflow.start_run(run_name="Random Forest Hyper-parameter Optimization", nested=True):
    best_params_rf = fmin(
        fn=objective_rf,
        space=search_space_rf,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )

    best_params_rf["max_depth"] = int(best_params_rf["max_depth"])
    best_params_rf["min_samples_split"] = int(best_params_rf["min_samples_split"])
    mlflow.log_params(best_params_rf)

    # Entrenar el modelo final con los mejores parámetros
    rf_final = RandomForestRegressor(**best_params_rf)
    rf_final.fit(X_train, y_train)

    # Predicción y cálculo de RMSE
    y_pred_rf = rf_final.predict(X_val)
    rmse_rf = root_mean_squared_error(y_val, y_pred_rf)
    mlflow.log_metric("rmse_final", rmse_rf)
    
    # Guardar el modelo
    mlflow.sklearn.log_model(rf_final, artifact_path="model_rf")

    # Guardar el preprocesador
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")


100%|██████████| 10/10 [04:32<00:00, 27.27s/trial, best loss: 5.358797747712526]


2024/09/20 15:11:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [26]:
import pandas as pd

# Convertir a DataFrame
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)


In [15]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
import numpy as np

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Habilitar el autologging
mlflow.sklearn.autolog()

def objective_rf(params):
    params['n_estimators'] = int(params['n_estimators'])  
    params['max_depth'] = int(params['max_depth'])  
    params['min_samples_split'] = int(params['min_samples_split'])
    
    with mlflow.start_run(nested=True):
        mlflow.log_params(params)
        
        # Crea y entrena el modelo
        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)

        # Predicción en el conjunto de validación
        y_pred = rf.predict(X_val)

        # Calcular RMSE
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

# Espacio de búsqueda de hiperparámetros para Random Forest
search_space_rf = {
    'n_estimators': hp.choice('n_estimators', [50, 100, 150]),
    'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
    'random_state': 42
}

# Optimización con Hyperopt
with mlflow.start_run(run_name="Random Forest Hyper-parameter Optimization", nested=True):
    best_params_rf = fmin(
        fn=objective_rf,
        space=search_space_rf,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )

    best_params_rf["max_depth"] = int(best_params_rf["max_depth"])
    best_params_rf["min_samples_split"] = int(best_params_rf["min_samples_split"])

    # Entrenar el modelo final con los mejores parámetros (esto ahora se autologea)
    rf_final = RandomForestRegressor(**best_params_rf)
    rf_final.fit(X_train, y_train)

    # Predicción y cálculo de RMSE
    y_pred_rf = rf_final.predict(X_val)
    rmse_rf = root_mean_squared_error(y_val, y_pred_rf)



  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/09/20 15:59:12 WARNING mlflow.data.pandas_dataset: Failed to infer schema for Pandas dataset. Exception: Failed to infer schema for pandas.Series 0        <Compressed Sparse Row sparse matrix of dtype ...
1        <Compressed Sparse Row sparse matrix of dtype ...
2        <Compressed Sparse Row sparse matrix of dtype ...
3        <Compressed Sparse Row sparse matrix of dtype ...
4        <Compressed Sparse Row sparse matrix of dtype ...
                               ...                        
54368    <Compressed Sparse Row sparse matrix of dtype ...
54369    <Compressed Sparse Row sparse matrix of dtype ...
54370    <Compressed Sparse Row sparse matrix of dtype ...
54371    <Compressed Sparse Row sparse matrix of dtype ...
54372    <Compressed Sparse Row sparse matrix of dtype ...
Name: 0, Length: 54373, dtype: object. Error: Data <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (1, 5093)>
  Coords	Values
  (0, 2065)	1.0
  (0, 5092)	1.98

  0%|          | 0/10 [00:01<?, ?trial/s, best loss=?]


2024/09/20 15:59:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest Hyper-parameter Optimization at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/0/runs/b738e05a99144ec0b8627e07cf298d45.
2024/09/20 15:59:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/0.


ValueError: setting an array element with a sequence.

## Gradient Boosting

In [37]:
from sklearn.ensemble import GradientBoostingRegressor

# Define la función objetivo para Gradient Boosting
def objective_gb(params):
    params['max_depth'] = int(params['max_depth'])  
    params['min_samples_split'] = int(params['min_samples_split'])  
    with mlflow.start_run(nested=True):
        mlflow.log_params(params)

        gb = GradientBoostingRegressor(**params)
        gb.fit(X_train, y_train)

        y_pred = gb.predict(X_val)

        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}


In [38]:
# Espacio de búsqueda de hiperparámetros para Gradient Boosting
search_space_gb = {
    'n_estimators': hp.choice('n_estimators', [100, 200]),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
    'random_state': 42
}

# Optimización con Hyperopt
with mlflow.start_run(run_name="Gradient Boosting Hyper-parameter Optimization", nested=True):
    best_params_gb = fmin(
        fn=objective_gb,
        space=search_space_gb,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )

    best_params_gb["max_depth"] = int(best_params_gb["max_depth"])
    best_params_gb["min_samples_split"] = int(best_params_gb["min_samples_split"])
    mlflow.log_params(best_params_gb)

    # Entrenar el modelo final con los mejores parámetros
    gb_final = GradientBoostingRegressor(**best_params_gb)
    gb_final.fit(X_train, y_train)

    # Predicción y cálculo de RMSE
    y_pred_gb = gb_final.predict(X_val)
    rmse_gb = root_mean_squared_error(y_val, y_pred_gb)
    mlflow.log_metric("rmse_final", rmse_gb)

    # Guardar el modelo
    mlflow.sklearn.log_model(gb_final, artifact_path="model_gb")

    # Guardar el preprocesador
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")


100%|██████████| 10/10 [01:34<00:00,  9.41s/trial, best loss: 5.203524379069028]


2024/09/20 15:15:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [39]:
best_params_gb

{'learning_rate': np.float64(0.4548440622970818),
 'max_depth': 8,
 'min_samples_split': 8,
 'n_estimators': np.int64(1)}

In [40]:
# Entrenar el modelo final con los mejores parámetros
rf_final = RandomForestRegressor(**best_params_rf)
rf_final.fit(X_train, y_train)

# Calcular RMSE en el conjunto de validación
y_pred_rf = rf_final.predict(X_val)
rmse_rf = root_mean_squared_error(y_val, y_pred_rf)


In [13]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as PacoTinoco

Initialized MLflow to track repo "PacoTinoco/nyc-taxi-time-prediction"

Repository PacoTinoco/nyc-taxi-time-prediction initialized!

https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/9e991ac0af45417cb6a8c46fbaf1900e', creation_time=1726630091495, experiment_id='0', last_update_time=1726630091495, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("PacoTinoco/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

## test
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)


Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

## LAST CHANCE

In [32]:
import mlflow
from hyperopt import fmin, tpe, hp, Trials
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pathlib
import pickle

mlflow.sklearn.autolog()  # Autolog para Random Forest

with mlflow.start_run(run_name="Random Forest Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': hp.quniform('max_depth', 4, 100, 1),
        'n_estimators': hp.quniform('n_estimators', 50, 500, 1),
        'min_samples_split': hp.quniform('min_samples_split', 2, 20, 1),
        'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 20, 1),
    }
    
    def objective(params):
        with mlflow.start_run(nested=True):
            # Log parameters
            mlflow.log_params(params)
            
            # Convert params to integers where necessary
            max_depth = int(params['max_depth'])
            n_estimators = int(params['n_estimators'])
            min_samples_split = int(params['min_samples_split'])
            min_samples_leaf = int(params['min_samples_leaf'])
            
            # Train model
            model = RandomForestRegressor(
                max_depth=max_depth,
                n_estimators=n_estimators,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                random_state=42
            )
            model.fit(X_train, y_train)  # Entrenar el modelo
            
            # Predict in the val dataset
            y_pred = model.predict(X_val)  # Predecir en el conjunto de validación
            
            # Calculate metric
            rmse = mean_squared_error(y_val, y_pred, squared=False)  # Calcular RMSE
            
            # Log performance metric
            mlflow.log_metric("rmse", rmse)

        return {'loss': rmse, 'status': STATUS_OK}
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["n_estimators"] = int(best_params["n_estimators"])
    best_params["min_samples_split"] = int(best_params["min_samples_split"])
    best_params["min_samples_leaf"] = int(best_params["min_samples_leaf"])
    
    mlflow.log_params(best_params)
    
    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "random_forest",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance with best parameters
    final_model = RandomForestRegressor(
        max_depth=best_params['max_depth'],
        n_estimators=best_params['n_estimators'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        random_state=42
    )
    
    final_model.fit(X_train, y_train)
    
    y_pred = final_model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/09/20 16:38:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID cc2e0a9ba39f413c82cb8a6c9d97f055. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/20 16:38:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run lyrical-jay-811 at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/cc2e0a9ba39f413c82cb8a6c9d97f055.

2024/09/20 16:38:48 INFO ml

 10%|█         | 1/10 [00:11<01:46, 11.80s/trial, best loss: 22.761197024718637]

2024/09/20 16:38:56 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 01d740add175409182ca6b994c6f2e61. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/20 16:38:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run orderly-roo-307 at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/01d740add175409182ca6b994c6f2e61.

2024/09/20 16:38:56 INFO ml

 20%|██        | 2/10 [00:19<01:15,  9.39s/trial, best loss: 22.761197024718637]

...

2024/09/20 16:39:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID d1bb432c88bd48439cc47a0f96a0dbaf. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/20 16:39:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run merciful-seal-773 at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/d1bb432c88bd48439cc47a0f96a0dbaf.

2024/09/20 16:39:48 INFO 

 90%|█████████ | 9/10 [01:11<00:07,  7.53s/trial, best loss: 22.761197024718637]

2024/09/20 16:39:56 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 17ef3d6050f840c0838abe04281f3a22. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/20 16:39:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run shivering-rook-891 at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/17ef3d6050f840c0838abe04281f3a22.

2024/09/20 16:39:56 INFO

100%|██████████| 10/10 [01:19<00:00,  7.95s/trial, best loss: 22.761197024718637]


C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/09/20 16:40:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest Hyper-parameter Optimization at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/f6994d345afd406e8b3ed819224831fc.
2024/09/20 16:40:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1.


In [29]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)


(54373, 1)
(51497, 1)
(54373,)
(51497,)


In [30]:
import pandas as pd

X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)


In [31]:
from sklearn.datasets import make_regression

X_train, y_train = make_regression(n_samples=100, n_features=1, noise=0.1)
X_val, y_val = make_regression(n_samples=50, n_features=1, noise=0.1)


In [33]:
from hyperopt import fmin, tpe, hp, Trials
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import pathlib

mlflow.sklearn.autolog()  # Autolog para Gradient Boosting

with mlflow.start_run(run_name="Gradient Boosting Hyper-parameter Optimization", nested=True):
    search_space = {
        'n_estimators': hp.quniform('n_estimators', 50, 500, 1),
        'max_depth': hp.quniform('max_depth', 1, 10, 1),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'min_samples_split': hp.quniform('min_samples_split', 2, 20, 1),
        'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 20, 1),
    }
    
    def objective(params):
        with mlflow.start_run(nested=True):
            # Log parameters
            mlflow.log_params(params)
            
            # Convert params to enteros donde sea necesario
            n_estimators = int(params['n_estimators'])
            max_depth = int(params['max_depth'])
            min_samples_split = int(params['min_samples_split'])
            min_samples_leaf = int(params['min_samples_leaf'])
            
            # Entrenar el modelo
            model = GradientBoostingRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=params['learning_rate'],
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                random_state=42
            )
            model.fit(X_train, y_train)  # Entrenar el modelo
            
            # Predecir en el conjunto de validación
            y_pred = model.predict(X_val)  # Predecir
            
            # Calcular métrica
            rmse = mean_squared_error(y_val, y_pred, squared=False)  # Calcular RMSE
            
            # Log de la métrica de rendimiento
            mlflow.log_metric("rmse", rmse)

        return {'loss': rmse, 'status': STATUS_OK}
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    
    best_params["n_estimators"] = int(best_params["n_estimators"])
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["min_samples_split"] = int(best_params["min_samples_split"])
    best_params["min_samples_leaf"] = int(best_params["min_samples_leaf"])
    
    mlflow.log_params(best_params)
    
    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "gradient_boosting",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance with best parameters
    final_model = GradientBoostingRegressor(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        learning_rate=params['learning_rate'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        random_state=42
    )
    
    final_model.fit(X_train, y_train)
    
    y_pred = final_model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")


  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/09/20 16:44:02 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 77b5a19e3d154c17acfb0e3340c12bdb. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/20 16:44:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run popular-sponge-820 at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/77b5a19e3d154c17acfb0e3340c12bdb.

2024/09/20 16:44:02 INFO

 10%|█         | 1/10 [00:08<01:12,  8.02s/trial, best loss: 29.35323637400974]

2024/09/20 16:44:09 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 63dd328fc64645af88f8141a061cac25. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/20 16:44:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run peaceful-cod-133 at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/63dd328fc64645af88f8141a061cac25.

2024/09/20 16:44:10 INFO m

 20%|██        | 2/10 [00:15<01:02,  7.81s/trial, best loss: 29.35323637400974]

...

 90%|█████████ | 9/10 [01:06<00:07,  7.22s/trial, best loss: 19.51307448420063]

2024/09/20 16:45:07 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 80c7e0685ee64ff8834843ea0827ec83. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Response: {\'error_code\': \'INVALID_PARAMETER_VALUE\'}")]')]

C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/09/20 16:45:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run painted-asp-604 at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/80c7e0685ee64ff8834843ea0827ec83.

2024/09/20 16:45:08 INFO ml

100%|██████████| 10/10 [01:13<00:00,  7.36s/trial, best loss: 19.51307448420063]


2024/09/20 16:45:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run Gradient Boosting Hyper-parameter Optimization at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/2a55c3e50a68491ab99aff27f3ad3df0.
2024/09/20 16:45:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1.


NameError: name 'params' is not defined

## Registrar el modelo en el Model Registry con alias challenger

In [56]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model2"
)

Successfully registered model 'nyc-taxi-model2'.
2024/09/20 17:25:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model2, version 1
Created version '1' of model 'nyc-taxi-model2'.


In [60]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "challenger"
date = datetime.today()
model_version = "2"

# create "challenger" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['challenger'], creation_timestamp=1726872572864, current_stage='None', description=('The model version 2 was transitioned to challenger on 2024-09-20 '
 '17:28:17.407310'), last_updated_timestamp=1726874897194, name='nyc-taxi-model', run_id='2962604a65044516a24ce15448e713b5', run_link='', source='mlflow-artifacts:/92863f65ec68402babfe2d09ddebc317/2962604a65044516a24ce15448e713b5/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [35]:
!curl -o ../data/green_tripdata_2024-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 50 1340k   50  670k    0     0   632k      0  0:00:02  0:00:01  0:00:01  634k
100 1340k  100 1340k    0     0   690k      0  0:00:01  0:00:01 --:--:--  691k


In [36]:
import os 
# Descargar el archivo de datos
os.system("curl -o ../data/green_tripdata_2024-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet")


0

In [37]:
# Registrar el archivo en MLflow
with mlflow.start_run(run_name="Download March 2024 Green Trip Data"):
    mlflow.log_artifact("../data/green_tripdata_2024-03.parquet", artifact_path="data")

2024/09/20 16:53:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run Download March 2024 Green Trip Data at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/9221d49c812a4dd7b5245b5399c3b710.
2024/09/20 16:53:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1.


In [51]:
import pandas as pd

# Cargar el conjunto de datos
data_path = "../data/green_tripdata_2024-03.parquet"
df = pd.read_parquet(data_path)

# Verificar la carga de datos
print(df.head())


   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2024-03-01 00:10:52   2024-03-01 00:26:12                  N   
1         2  2024-03-01 00:22:21   2024-03-01 00:35:15                  N   
2         2  2024-03-01 00:45:27   2024-03-01 01:04:32                  N   
3         1  2024-03-01 00:02:00   2024-03-01 00:23:45                  N   
4         2  2024-03-01 00:16:45   2024-03-01 00:23:25                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0         1.0           129           226              1.0           1.72   
1         1.0           130           218              1.0           3.25   
2         1.0           255           107              2.0           4.58   
3         1.0           181            71              1.0           0.00   
4         1.0            95           135              1.0           1.15   

   fare_amount  extra  mta_tax  tip_amount  tolls_amount  ehail_fee  \
0  

In [52]:
df.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')

In [53]:
# Convertir columnas de fecha a datetime
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

In [54]:
# Seleccionar características relevantes
features = [
    'VendorID', 'RatecodeID', 'PULocationID', 'DOLocationID',
    'passenger_count', 'trip_distance', 'fare_amount', 'extra',
    'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee',
    'improvement_surcharge'
]

X = df[features]
y = df['total_amount']

In [48]:
# Manejo de valores nulos (si es necesario)
X.dropna(inplace=True)
y = y[X.index]

C:\Users\Francisco\AppData\Local\Temp\ipykernel_23440\2180193586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)


In [55]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [65]:
from xgboost import XGBRegressor 
model_champion = XGBRegressor(
    max_depth=86,
    learning_rate=0.96,
    n_estimators=100,
    objective='reg:squarederror'
)


In [66]:
from sklearn.ensemble import RandomForestRegressor  
model_challenger = RandomForestRegressor(
    n_estimators=149,
    max_depth=27,
    random_state=42
)


In [67]:
from sklearn.metrics import mean_squared_error

# Entrenar el modelo champion
model_champion.fit(X_train, y_train)

# Predecir y calcular métrica para el modelo champion
y_pred_champion = model_champion.predict(X_val)
rmse_champion = mean_squared_error(y_val, y_pred_champion, squared=False)

# Entrenar el modelo challenger
model_challenger.fit(X_train, y_train)

# Predecir y calcular métrica para el modelo challenger
y_pred_challenger = model_challenger.predict(X_val)
rmse_challenger = mean_squared_error(y_val, y_pred_challenger, squared=False)

# Imprimir métricas
print(f"RMSE Champion Model: {rmse_champion}")
print(f"RMSE Challenger Model: {rmse_challenger}")


C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/09/20 17:41:17 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6ec271e37c1747f7af75067b5da2c3e0', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/09/20 17:41:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Francisco\Downloads\proyectoCienciaDeDatos2\nyc-taxi-time-prediction\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, i

RMSE Champion Model: 3.805264765828367
RMSE Challenger Model: 3.0622049182672364


In [68]:
import mlflow

with mlflow.start_run(run_name="Model Comparison March 2024"):
    mlflow.log_metric("rmse_champion", rmse_champion)
    mlflow.log_metric("rmse_challenger", rmse_challenger)


2024/09/20 17:43:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run Model Comparison March 2024 at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1/runs/37087d4b7ffa467db57b645e7aafbf97.
2024/09/20 17:43:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/PacoTinoco/nyc-taxi-time-prediction.mlflow/#/experiments/1.
